In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import Libraries

In [88]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Import Dataset

In [89]:
dataset = pd.read_csv('Multiple Linear Regression/kc_house_data.csv')
dataset.drop(columns=['id', 'date'], inplace=True)
#Exculde Date, Id, price column
# X = dataset.iloc[:, 3:].values
# y = dataset.iloc[:, 2].values
dataset

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


# Data preproessing

## Missing value

In [90]:
dataset.isna().sum()

price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

## Check Data types

In [91]:
print('Datatype:', dataset.dtypes)

Datatype: price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object


# Train the regression model

##Splitting the dataset into the Training set and Test set

In [92]:
from sklearn.model_selection import train_test_split
X = dataset.drop(columns=['price'])
y = dataset['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

##Training the Multiple Linear Regression model on the Training set

In [93]:
from sklearn.linear_model import LinearRegression
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

##Predicting the Test set results

In [94]:
y_pred = regression.predict(X_test)

##Model Accouracy

In [95]:
from sklearn import metrics
print("Training set score: {:.7f}".format(regression.score(X_train, y_train)))
print("Test set score: {:.7f}".format(regression.score(X_test, y_test)))
print("RMSE: {:.7f}".format(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))

Training set score: 0.7037209
Test set score: 0.6858691
RMSE: 232850.3620846


###### `After model training, we got less accuracy on training data and testing data which means model underfitted, will apply the outlier treatment and check the accuracy in below:


# Outlier treatment

In [96]:
outlier_data = pd.DataFrame(dataset)
''' Detection '''
# IQR
Q1 = np.percentile(outlier_data['price'], 25,interpolation = 'midpoint')

Q3 = np.percentile(outlier_data['price'], 75,interpolation = 'midpoint')
IQR = Q3 - Q1

print("Old Shape: ", outlier_data.shape)

# Upper bound
upper = np.where(outlier_data['price'] >= (Q3+1.5*IQR))
# Lower bound
lower = np.where(outlier_data['price'] <= (Q1-1.5*IQR))

''' Removing the Outliers '''
outlier_data.drop(upper[0], inplace = True)
outlier_data.drop(lower[0], inplace = True)

print("New Shape: ", outlier_data.shape)

Old Shape:  (21613, 19)
New Shape:  (20467, 19)


##Splitting the dataset into the Training set and Test set - after outlier removed

In [97]:
X1 = outlier_data.drop(columns=['price'])
y1 = outlier_data['price']
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.2, random_state = 1)


##Train the model after outlier treatment

In [98]:
regression_outlier = LinearRegression()
regression_outlier.fit(X1_train,y1_train)

LinearRegression()

##Predict test set result after outlier treatment

In [99]:
y1_pred = regression_outlier.predict(X1_test)

## Model Accuracy after outlier treatment

In [100]:
print("Training set score: {:.7f}".format(regression_outlier.score(X1_train, y1_train)))
print("Test set score: {:.7f}".format(regression_outlier.score(X1_test, y1_test)))
print("RMSE: {:.7f}".format(np.sqrt(metrics.mean_squared_error(y1_test, y1_pred))))

Training set score: 0.6962932
Test set score: 0.6982029
RMSE: 114885.2612469


#### Here has been got again less accuracy compared without outlier treatment accuracy. Threfore will train the model with most significant features via feature selection technique rather than train the model with all features.



# Feature selection technique

## Backward feature selection technique

In [101]:
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

###Splitting the dataset into the Training set and Test set - Backward feature selection

In [102]:
X2 = dataset.values[:,1:]
y2 = dataset.values[:,0]
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2, random_state = 1)

In [103]:
y2_train = y2_train.ravel()
y2_test = y2_test.ravel()

##  Train the model - Backward feature selection

In [104]:
regression_bw = LinearRegression()

# Build step forward feature selection
sfs1 = sfs(regression_bw,
           k_features=15,
           forward=False,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

# Perform SFFS
sfs1 = sfs1.fit(X2_train, y2_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py", line 95, in _check

In [105]:
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [106]:

# Build full model with selected features
nw = LinearRegression()
nw.fit(X2_train[:, feat_cols], y2_train)
y2_pred = nw.predict(X2_test[:, feat_cols])

print("Training set score: {:.7f}".format(nw.score(X2_train[:, feat_cols], y2_train)))
print("Test set score: {:.7f}".format(nw.score(X2_test[:, feat_cols], y2_test)))
print("RMSE: {:.7f}".format(np.sqrt(metrics.mean_squared_error(y2_test, y2_pred))))


Training set score: 0.6992740
Test set score: 0.6812908
RMSE: 234541.0702199


#Conclusion
#### Above performed three methods, tried to incrase model accuracy however, it not give good accuracy. Therefore, we should consider different regression algorithum for this use case.